# First Prompt

In [1]:
import sys
sys.path.append('/home/olemke/workspace/code/spot-compose-v2/source')

In [1]:
import json
import os
import shutil
import zipfile

import numpy as np
import torch

import clip
import open3d as o3d
import requests
from urllib3.exceptions import ReadTimeoutError
from utils import recursive_config
from utils.docker_interfaces.docker_communication import _get_content
from utils.recursive_config import Config


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
config = Config()

MIN_VOLUME = 0.01
MIN_SIDE = 0.01
MIN_SCORE = 0.5
INTERMEDIATE_VISUALIZATIONS = True

In [15]:
pcd_name = config["pre_scanned_graphs"]["high_res"]
base_path = config.get_subpath("openmask_features")
feat_path = os.path.join(base_path, pcd_name, "clip_features_comp.npy")
mask_path = os.path.join(base_path, pcd_name, "scene_MASKS_comp.npy")
score_path = os.path.join(base_path, pcd_name, "scene_SCORES_comp.npy")
class_path = os.path.join(base_path, pcd_name, "scene_CLASSES_comp.npy")
pcd_path = os.path.join(
    config.get_subpath("aligned_point_clouds"), pcd_name, "scene.ply"
)

features = np.load(feat_path)
masks = np.load(mask_path)
scores = np.load(score_path)
classes = np.load(class_path)

In [16]:
pass_score_bool = scores > MIN_SCORE
masks = masks[pass_score_bool]
scores = scores[pass_score_bool]
features = features[pass_score_bool]

In [17]:
masks = masks.astype(bool)
nr_objects = masks.shape[0]
pcd = o3d.io.read_point_cloud(str(pcd_path))
points = np.asarray(pcd.points)

# get the mins and maxes per object
maskss = np.repeat(masks[..., np.newaxis], 3, axis=2)
pointss = np.repeat(points[np.newaxis, ...], nr_objects, axis=0)
masked_pointss = np.ma.array(
    pointss, mask=~maskss
)  # True means masked, opposite meaning therefore ~
bbox_mins = masked_pointss.min(axis=1)
bbox_maxs = masked_pointss.max(axis=1)

bbox_sides  = bbox_maxs - bbox_mins
bbox_above_side_length = np.all(bbox_sides > MIN_SIDE, axis=-1)
bbox_volumes = np.prod(bbox_sides, axis=-1)
bbox_above_min_volume = bbox_volumes > MIN_VOLUME
bbox_mins, bbox_maxs = (
    bbox_mins[bbox_above_side_length & bbox_above_min_volume],
    bbox_maxs[bbox_above_side_length & bbox_above_min_volume],
)
nr_bboxes = np.sum(bbox_above_min_volume)

if INTERMEDIATE_VISUALIZATIONS:
    aabbs = []
    for i in range(nr_bboxes):
        bbox_min = bbox_mins[i]
        bbox_max = bbox_maxs[i]

        # Create an axis-aligned bounding box
        aabb = o3d.geometry.AxisAlignedBoundingBox(
            min_bound=bbox_min, max_bound=bbox_max
        )
        aabbs.append(aabb)

    # Optionally, visualize the bounding box
    o3d.visualization.draw_geometries([pcd, *aabbs])